# Evaluation for the fourth model

In [ ]:
from pathlib import Path
import numpy as np
import torch
from torch.utils.data import DataLoader

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

In [ ]:
import sys; sys.path.insert(0, '/mnt/src')

## Read Datasets from .csv

In [ ]:
from utils.file_io import read_parallel_trajectory_datasets

In [ ]:
data_path = Path("/mnt/data/")

feature_columns = [
    'left_boom_base_yaw_joint', 
    'left_boom_base_pitch_joint',
    'left_boom_main_prismatic_joint',
    'left_boom_second_roll_joint',
    'left_boom_second_yaw_joint',
    'left_boom_top_pitch_joint',
    'left_boom_ee_joint',
    'cable1_property(length,youngsmodule(bend,twist))',
    'cable2_property(length,youngsmodule(bend,twist))',
    'cable3_property(length,youngsmodule(bend,twist))'
]

label_features = [
    ('cable1_lowest_point', np.array([2], dtype=np.int64)),
    ('cable2_lowest_point', np.array([2], dtype=np.int64)),
    ('cable3_lowest_point', np.array([2], dtype=np.int64))
]

normalized_features = [
    ('cable1_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable2_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64)),
    ('cable3_property(length,youngsmodule(bend,twist))', np.array([1,2], dtype=np.int64))
]

In [ ]:
data_path = Path("/mnt/data")
train_set, test_set, validation_set, visualization_set = read_parallel_trajectory_datasets(data_path, 0.8, 0.15, 0.045, 0.005, 64, 
                                                                    feature_columns=feature_columns, label_features=label_features, normalized_features=normalized_features)

In [ ]:
features, labels = train_set[0] 
print(features.shape, labels.shape)
input_shape, output_shape = features.shape[-1], labels.shape[-1]
num_parallel_trajectories = features.shape[0]
print(f"Data shape {input_shape} / {output_shape} of total {len(train_set) + len(validation_set)} data rows!")

## Load parameter, functions and dataloader

In [ ]:
import os
import ast
from dotenv import load_dotenv

from utils.file_io import define_dataloader_from_subset
from utils.activation import get_activation

In [ ]:
model_path = Path("/mnt/models/two_stage/")

In [ ]:
dotenv_path = model_path / ".env"
load_dotenv(dotenv_path=dotenv_path)

encoder_batch_size = int(os.getenv("ENCODER_BATCH_SIZE"))
encoder_optimizer = os.getenv("ENCODER_OPTIMIZER")
encoder_activation = os.getenv("ENCODER_ACTIVATION")
num_encoder_layer = int(os.getenv("NUM_ENCODER_LAYER"))
pos_encoder_dropout = float(os.getenv("POS_ENCODER_DROPOUT"))
encoder_transformer_dropout = float(os.getenv("ENCODER_TRANSFORMER_DROPOUT"))
encoder_feedforward_dim = int(os.getenv("ENCODER_FEEDFORWARD_DIM"))
encoder_warmup_steps = int(os.getenv("ENCODER_WARMUP_STEPS"))
encoder_model_dim_num_heads_projection = ast.literal_eval(os.getenv("ENCODER_MODEL_DIM_NUM_HEADS_PROJECTION"))
encoder_num_epochs = int(os.getenv("ENCODER_NUM_EPOCHS"))

decoder_batch_size = int(os.getenv("DECODER_BATCH_SIZE"))
decoder_optimizer = os.getenv("DECODER_OPTIMIZER")
decoder_activation = os.getenv("DECODER_ACTIVATION")
num_decoder_layer = int(os.getenv("NUM_DECODER_LAYER"))
decoder_transformer_dropout = float(os.getenv("DECODER_TRANSFORMER_DROPOUT"))
decoder_feedforward_dim = int(os.getenv("DECODER_FEEDFORWARD_DIM"))
decoder_warmup_steps = int(os.getenv("DECODER_WARMUP_STEPS"))
decoder_model_dim_num_heads = ast.literal_eval(os.getenv("DECODER_MODEL_DIM_NUM_HEADS"))
decoder_num_epochs = int(os.getenv("DECODER_NUM_EPOCHS"))

In [ ]:
encoder_activation = get_activation(encoder_activation)
decoder_activation = get_activation(decoder_activation)

## Train downprojection

In [ ]:
from umap import UMAP

from models.transformer import train_downprojection
from utils.file_io import save_downprojection

In [ ]:
encoder_train_dataloader, encoder_validation_dataloader, encoder_test_dataloader = define_dataloader_from_subset(train_set, validation_set, test_set, batch_size=encoder_batch_size)

In [ ]:
downprojection = UMAP(n_components=3, n_neighbors=encoder_model_dim_num_heads_projection[2])
downprojection = train_downprojection(downprojection, encoder_train_dataloader)
save_downprojection(downprojection, model_path / "downprojection.sav")

## Train Encoder Model

In [ ]:
from models.parallel_encoder_model import ParallelEncoderModel
from models import parallel_encoder_model
from utils.loss_functions import get_loss_function
from utils.optimizer import get_optimizer_function, get_learning_rate_scheduler
from utils.early_stopping import EarlyStopping

In [ ]:
parallel_encoder = ParallelEncoderModel(
    num_decoders = num_parallel_trajectories, 
    num_heads = encoder_model_dim_num_heads_projection[1], 
    model_dim = encoder_model_dim_num_heads_projection[0], 
    feedforward_hidden_dim = encoder_feedforward_dim, 
    output_dim = output_shape, 
    num_encoder_layers = num_encoder_layer,
    transformer_dropout = encoder_transformer_dropout, 
    pos_encoder_dropout = pos_encoder_dropout, 
    activation = encoder_activation, 
    projection_function = downprojection
)

In [ ]:
optimizer = get_optimizer_function(encoder_optimizer, parallel_encoder, 1)
lr_scheduler = get_learning_rate_scheduler(optimizer, encoder_model_dim_num_heads_projection[0], encoder_warmup_steps)
loss_function = get_loss_function()

In [ ]:
early_stopping = EarlyStopping(25)

In [ ]:
encoder_model_dir = model_path / "encoder"
encoder_model_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
_, encoder_validation_losses = parallel_encoder_model.train(
    encoder_num_epochs, 
    encoder_train_dataloader, 
    encoder_validation_dataloader, 
    parallel_encoder, 
    loss_function, 
    optimizer, 
    lr_scheduler, 
    encoder_model_dir, 
    device, 
    early_stopping=early_stopping
)

# Train Encoder-Decoder Model

In [ ]:
from models.parallel_decoder_model import TransformerDecoderModel, TransformerModel
from models import parallel_decoder_model

In [ ]:
model_state_dict = torch.load(model_path / "encoder" / "checkpoint.pt")
parallel_encoder.load_state_dict(model_state_dict)

encoder = parallel_encoder.encoder

In [ ]:
decoder_train_dataloader, decoder_validation_dataloader, decoder_test_dataloader = define_dataloader_from_subset(train_set, validation_set, test_set, batch_size=decoder_batch_size)

In [ ]:
decoder = TransformerDecoderModel(
    model_dim = decoder_model_dim_num_heads[0],
    num_heads = decoder_model_dim_num_heads[1],
    feedforward_dim = decoder_feedforward_dim,
    num_decoder_layers = num_decoder_layer,
    pos_encoder = encoder.pos_encoder,
    transformer_dropout = decoder_transformer_dropout,
    activation = decoder_activation
)

In [ ]:
model = TransformerModel(encoder, decoder)

In [ ]:
optimizer = get_optimizer_function(decoder_optimizer, model, 1)
lr_scheduler = get_learning_rate_scheduler(optimizer, decoder_model_dim_num_heads[0], decoder_warmup_steps)
loss_function = get_loss_function()

In [ ]:
early_stopping = EarlyStopping(25)

In [ ]:
_, validation_losses = parallel_decoder_model.train(
    epochs = decoder_num_epochs,
    train_dataloader = decoder_train_dataloader,
    validation_dataloader = decoder_validation_dataloader,
    model = model,
    loss_function = loss_function,
    optimizer = optimizer,
    lr_scheduler = lr_scheduler,
    checkpoint_path = model_path,
    device = device,
    early_stopping = early_stopping
) 

## Evaluation

In [ ]:
from utils.evaluation import compute_losses_from, compute_sliding_window_predictions
from utils.visualization import create_plot_for_dimensions, create_inference_time_plot, create_validation_loss_plot

In [ ]:
model_state_dict = torch.load(model_path / "checkpoint.pt")
model.load_state_dict(model_state_dict)

In [ ]:
model.eval()

## 1. Analysis: Compute loss and average inference time on test

In [ ]:
y, y_true, inference_times = compute_sliding_window_predictions(decoder_test_dataloader, model, 'cpu')
test_losses = compute_losses_from(y, y_true, get_loss_function())
print(f"The mean squared error of the loaded model on test is: {test_losses.mean()}")
print(f"The average inference time of the loaded model on test is: {inference_times.mean()} seconds.")

## 2. Analysis: Show Cable lowest points

In [ ]:
visualization_dataloader = DataLoader(visualization_set, batch_size=1, shuffle=False)

In [ ]:
y, y_true, inference_times = compute_sliding_window_predictions(visualization_dataloader, model, 'cpu')

In [ ]:
plot = create_plot_for_dimensions(y.numpy(), y_true.numpy(), size=5)
plot.savefig(model_path / "predictions.png")
plot.show()

## 3. Analysis: Show inference times

In [ ]:
plot = create_inference_time_plot(inference_times.numpy())
plot.savefig(model_path / "inference_times.png")
plot.show()

## 4. Analysis: Show validation loss over time

In [ ]:
plot = create_validation_loss_plot(validation_losses)
plot.savefig(model_path / "losses.png")
plot.show()

In [ ]:
plot = create_validation_loss_plot(encoder_validation_losses)
plot.savefig(model_path / "encoder_losses.png")
plot.show()